# Analyze influences among music composers since the middle-ages
# !!! Ici inclure une photo de notre output !!!
## Introduction
This project will analyze classical music composer's data parsed from wikipedia. We focus the notion of influence to detect the most influential composers


In [23]:
import wikipedia
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [2]:
# Fetch wikipedia page from a page name. Tries with and without autosuggest. Return None if no page was found.
def get_wiki_page(link):
    try:
        composer = wikipedia.page(title=link, auto_suggest=False)
        return composer
    except:
        pass
    try:
        composer = wikipedia.page(title=link, auto_suggest=True)
        return composer
    except:
        return None

# Bulk addition of a tag to the information dict.
def add_information(info_dict, name_set, tag_key, tag_value):
    for name in name_set:
        if name in info_dict:
            info_dict[name][tag_key] = tag_value
        else:
            info_dict[name] = {}
            info_dict[name][tag_key] = tag_value

# composer -> information dict (= birthplace -> Pittsburgh ...)
composer_information = {}
# Set of wikipedia links to composer pages
links_set = set()

composers = set(get_wiki_page("List_of_medieval_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'medieval')

composers = set(get_wiki_page("List_of_Baroque_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'baroque')

composers = set(get_wiki_page("List_of_Classical-era_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'classical')

composers = set(get_wiki_page("List_of_Romantic-era_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'romantic')

composers = set(get_wiki_page("List_of_20th-century_classical_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'modern')

composers = set(get_wiki_page("List_of_21st-century_classical_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'contemporary')

links = list(links_set)
relations = {}
composer_count = 0
for link in links:
    try:
        # Obtain wikipedia page corresponding to the composer
        composer = get_wiki_page(link)
        if composer == None:
            continue

        # Check if the page indeed relates to a composer
        is_composer = False
        for category in composer.categories:
            if 'composer' in category.lower():
                is_composer = True
        if not is_composer:
            continue

        # Only keep composers for which a birthplace and date of birth was provided
        has_birthplace = False
        has_dob = False
        for c in composer.categories:
            if c.startswith('People from '):
                composer_information[link]['birthplace'] = c[len('People from '):]
                has_birthplace = True
            if c.endswith(' births'):
                composer_information[link]['dob'] = c[:-len(' births')]
                has_dob = True
        if not (has_birthplace and has_dob):
            del composer_information[link]
            continue

        # If all the filters are passed, add to relations list
        relations[link] = composer.links
        composer_count += 1
        if (composer_count == 1 or composer_count % 10 == 0):
            print("composer_count: " + str(composer_count))
            
    except KeyboardInterrupt:
        print("Exiting. [KeyboardInterrupt]")
    except :
        print("ERROR " + link + " -----------------------------")

composer_count: 1
composer_count: 10
composer_count: 20
composer_count: 30
composer_count: 40
composer_count: 50
composer_count: 60
composer_count: 70
composer_count: 80
composer_count: 90
composer_count: 100
composer_count: 110
composer_count: 120
composer_count: 130
composer_count: 140
composer_count: 150
composer_count: 160
composer_count: 170
composer_count: 180
composer_count: 190
composer_count: 200
composer_count: 210
composer_count: 220
composer_count: 230
composer_count: 240
composer_count: 250
composer_count: 260
composer_count: 270
composer_count: 280
composer_count: 290
composer_count: 300
composer_count: 310
composer_count: 320
composer_count: 330
composer_count: 340
composer_count: 350
composer_count: 360
composer_count: 370
composer_count: 380
composer_count: 390
composer_count: 400
composer_count: 410
composer_count: 420
composer_count: 430
composer_count: 440
composer_count: 450
composer_count: 460
composer_count: 470
composer_count: 480
composer_count: 490
composer_co

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


composer_count: 1530
composer_count: 1540
composer_count: 1550
composer_count: 1560
composer_count: 1570
composer_count: 1580
composer_count: 1590
composer_count: 1600
composer_count: 1610
composer_count: 1620
composer_count: 1630
composer_count: 1640
composer_count: 1650
composer_count: 1660
composer_count: 1670
composer_count: 1680
composer_count: 1690
composer_count: 1700
composer_count: 1710
composer_count: 1720
composer_count: 1730
composer_count: 1740
composer_count: 1750
composer_count: 1760
composer_count: 1770
composer_count: 1780
composer_count: 1790
composer_count: 1800
composer_count: 1810
composer_count: 1820
composer_count: 1830
composer_count: 1840
composer_count: 1850
composer_count: 1860
composer_count: 1870
composer_count: 1880
composer_count: 1890
composer_count: 1900
composer_count: 1910
composer_count: 1920
composer_count: 1930
composer_count: 1940
composer_count: 1950
composer_count: 1960
composer_count: 1970
composer_count: 1980
composer_count: 1990
composer_coun

In [4]:
# In the end we only consider the relations of composers with enough data available 
composer_information = {k: v for k, v in composer_information.items() if len(v) == 3}
allComposers = composer_information.keys()

In [5]:
def filterComposerRelationsOnly(relationsOfComposer, allComposers):
    return [relation for relation in relationsOfComposer if relation in allComposers]

relations = {k: filterComposerRelationsOnly(relationsOfComposer, allComposers) for k, relationsOfComposer in relations.items()}

In [6]:
file = open("follower_followee.txt","w") 
for followeeComposer,followedComposers in relations.items():
    for followedComposer in followedComposers:
        file.write(followeeComposer + "," + followedComposer + "\n")  
file.close() 

In [7]:
file = open('composer_information.txt','w')
for name, info in composer_information.items():
    file.write(name + '\t' + info['dob'] + '\t' + info['birthplace'] + '\t' + info['genre'] + '\n')
file.close()

In [25]:
birthplace_to_location = {}

In [26]:
geolocator = Nominatim()

timed_outs = 0
total = 0
for name, info in composer_information.items():
    if info['birthplace'] not in birthplace_to_location or birthplace_to_location[info['birthplace']] == None:
        try:
            total += 1
            if (total == 1 or total % 10 == 0):
                print(str(total) + ' processed with ' + str(timed_outs) + ' timeouts')  
            birthplace_to_location[info['birthplace']] = location = geolocator.geocode(name)
        except GeocoderTimedOut:
            timed_outs += 1
            print('timed out')
print('timed_outs: ' + str(timed_outs))
    

1 processed with 0 timeouts
timed out
10 processed with 1 timeouts
20 processed with 1 timeouts
timed out
30 processed with 2 timeouts
40 processed with 2 timeouts
timed out
50 processed with 3 timeouts
60 processed with 3 timeouts
70 processed with 3 timeouts
80 processed with 3 timeouts
90 processed with 3 timeouts
100 processed with 3 timeouts
110 processed with 3 timeouts
120 processed with 3 timeouts
130 processed with 3 timeouts
timed out
140 processed with 4 timeouts
150 processed with 4 timeouts
160 processed with 4 timeouts
timed out
170 processed with 5 timeouts
180 processed with 5 timeouts
190 processed with 5 timeouts
timed out
200 processed with 6 timeouts
210 processed with 6 timeouts
220 processed with 6 timeouts
230 processed with 6 timeouts
240 processed with 6 timeouts
250 processed with 6 timeouts
260 processed with 6 timeouts
270 processed with 6 timeouts
timed out
280 processed with 7 timeouts
290 processed with 7 timeouts
300 processed with 7 timeouts
310 process

In [29]:
timed_outs = 0
total = 0
for name, info in composer_information.items():
    if info['birthplace'] not in birthplace_to_location or birthplace_to_location[info['birthplace']] == None:
        try:
            total += 1
            if (total == 1 or total % 10 == 0):
                print(str(total) + ' processed with ' + str(timed_outs) + ' timeouts')  
            birthplace_to_location[info['birthplace']] = location = geolocator.geocode(name)
        except GeocoderTimedOut:
            timed_outs += 1
            print('timed out')
print('timed_outs: ' + str(timed_outs))

1 processed with 0 timeouts
timed out
10 processed with 1 timeouts


KeyboardInterrupt: 

In [28]:
print(str(birthplace_to_location))

{'Auvergne': Location(Rue Hugues de Peirol, Les Farges, Clermont-Ferrand, Puy-de-Dôme, Auvergne-Rhône-Alpes, France métropolitaine, 63178, France, (45.7726919, 3.1279273, 0.0)), 'Cambrai': Location(Rue du Puits Grenon, Jargeau, Orléans, Loiret, Centre-Val de Loire, France métropolitaine, 45150, France, (47.8671844, 2.1213089, 0.0)), 'Orange, Vaucluse': None, 'Marseille': None, 'Pustertal': Location(Oswald-von-Wolkenstein, Gereuth, Brixen - Bressanone, Eisacktal - Valle Isarco, BZ, TAA, 39042, Italia, (46.7050763, 11.6563113, 0.0)), 'Alzey-Worms': Location(Hildegard-von-Bingen-Straße, Hollerstauden, Friedrichshofen-Hollerstauden, Ingolstadt, Obb, Bayern, 85049, Deutschland, (48.7640748, 11.3881139, 0.0)), 'Arras': Location(Adam de la Halle, Boulevard Admunsen, Meaux, Seine-et-Marne, Île-de-France, France métropolitaine, 77100, France, (48.954441, 2.907283, 0.0)), 'Tongeren': None, 'Champagne': None, 'Cividale del Friuli': Location(Cividale, Unione Terre di Pianura, BO, EMR, 40057, Itali

In [31]:
file = open('birthplace_to_location.txt','w')        
for birthplace, loc in birthplace_to_location.items():
    if loc != None:
        file.write(birthplace + '\t' + loc.raw['lat'] + '\t' + loc.raw['lon'] + '\t' + str(loc).split(',')[-1].strip() + '\n')
file.close()